# Rolling Windows Tutorial

This tutorial notebook demonstrates the basic pipeline for using the Lexos `rollingwindows` module. See the [README](README.me) file for the module's development status.

This tutorial has the following requirements:

- It should be run in a Python environment (v3.9 or above) with the Lexos API installed: `pip install lexos`. This will install most of the dependencies needed by `rollingwindows`.
- Run `pip install timer`. There may be one or two other Python packages that you will need to install if you encounter errors, but this is the only one currently known.
- You must have downloaded spaCy's small English language model: `python -m spacy download en_core_web_sm`. The tutorial assumes for demonstration purposes that you will use an English-language text with this model.
- A spaCy [`Doc`](https://spacy.io/api/doc) object created with this model. Although you can create one of your own using Lexos or spaCy, to save time the repo provides a pre-created `Doc` of Jane Austen's _Pride and Prejudice_, which can be loaded quickly.

Since `rollingwindows` has not yet been incorporated into the Lexos library, it is necessary to import it locally. The easiest way to do that is to set this notebook's `sys.path` to the folder containing the `rollingwindows` folder. To do this, modify the path in the cell below and then run the cell. Make sure that you are working in the environment containing Lexos.

In [ ]:
import sys
sys.path.insert(1, "C:/Users/YourName/Documents/")

## Load a spaCy `Doc`

Run the cell below to load the sample spaCy `Doc`. You can modify the path if necessary.

In [ ]:
path = "sample_docs/Austen_Pride"
model = "en_core_web_sm"

import spacy
from spacy.tokens import Doc
nlp = spacy.load(model)

with open(path, "rb") as f:
    doc_bytes = f.read()
doc = Doc(nlp.vocab).from_bytes(doc_bytes)

In the current version, processing times can be considerable for longer documents. So, for the purposes of demonstration, we will use only the first 10,000 tokens (approximately the first seven chapters).

In [ ]:
# Take a slice of the doc
doc = doc[0:3000].as_doc()

## Create a `RollingWindows` Object

The `RollingWindows` class helps you manage the workflow from importing and manipulating data through analysing the data and visualisation. The code below shows you how to create an instance of the class using the spaCy `Doc` and model configured in the previous cell.  

In [ ]:
import rollingwindows

rw = rollingwindows.RollingWindows(doc, model=model)

rw.metadata

The `RollingWindows` class has a `metadata` property that keeps track of the current state of the instance. As you do more work, `rw.metadata` will be updated with the settings you use.

### Setting Patterns

The `RollingWindows` class accepts and optional `patterns` keyword, which sets the patterns you wish to match when you perform your Rolling Windows analyses. Patterns can take the form of a string or regex pattern, a [spaCy `Matcher` rule](https://spacy.io/usage/rule-based-matching#matcher), or a list of multiple patterns. Here are some examples:

```python
rw = rollingwindows.RollingWindows(doc, model=model, patterns="love")

rw = rollingwindows.RollingWindows(doc, model=model, patterns=["love", "cheese"])

rw = rollingwindows.RollingWindows(
    doc,
    model=model,
    patterns=[{"LEMMA": "love"}, {"LEMMA": "cheese"}]
)
```

The last example will match all sequences in which any form of the word "cheese" follows any form of the word "love". In addition to matching exact character sequences or regex patterns, you have at your disposal all of the power of spaCy's `Matcher` class.

If you do not set the `patterns` when instantiating the class, you have the opportunity to do so later in the workflow.

## Set the Windows

Rolling Windows counts the number of matches to your patterns in sliding set of windows of a given length, incrementing the window one unit forward until the document is exhausted.

The first step is to generate these windows and assign them to your `RollingWindows` object. Window length may be counted in `characters`, `tokens` (meaning spacy [`Token`](https://spacy.io/api/token) objects), `lines`, or `sentences` (`sentences` are only possible if using a language model with a sentence segmenter).

To set the `RollingWindows` object's windows, call `rw.set_windows()` (assuming that you have assigned the `RollingWindows` object to the variable `rw`). In the example below, we generate a new window every 50 characters.


In [ ]:
rw.set_windows(1000, "tokens")

There are a number of optional parameters which can be used to modify the shape the windows (see the API documentation for details). The most important is the `filter` parameter, which allows you to filter your document before windows are generated. For instance, you may wish to include only tokens that are interpretable as words.

The `set_windows()` method creates an instance of the `Windows` class, which stores information about the windows, and assigns it to the `windows` attribute of your `RollingWindows` object.

> [!WARNING]  
> You can iterate through `rw.windows` but be aware that it is a generator which will be emptied in the process. If you need to access the windows again, you will need to re-run `set_windows()`.

## Calculate Rolling Averages

Rolling Windows statistics are calculated using calculator classes ("calculators"), of which there is currently one: `RWCalculator`.

The easiest way to calculate rolling averages is to import the `calculators` submodule and pass the `patterns` to detect, the type of `query` to make (in this case "averages"), and the id of the `rw_calculator` class to the `RollingWindows.calculate()` method:

```python
from rollingwindows import calculators
rw.calculate(patterns=["love", "marriage"], query="averages", calculator="rw_calculator")
```

Other `query` types are "counts" and "ratios".

> [!WARNING]  
> Documents longer than 10,000 tokens may take a long time to process. We are currently working on a way to make processing more efficient. In the meantime, you are encouraged to test with the smaller document generated above.

You can iterate through `rw.windows` but be aware that it is a generator which will be emptied in the process. If you need to access the windows again, you will need to re-run `set_windows()`.

In general, it is better to instantiate and configure a calculator and then pass that calculator object to the `RollingWindows.calculate()` method. This technique is shown below.

In [ ]:
from rollingwindows.calculators import RWCalculator

calculator = RWCalculator(
    patterns=["love", "marriage"],
    windows=rw.windows,
    query="averages"
)

rw.calculate(calculator=calculator)

Note that we are passing the windows from our `RollingWindows` object to our calculator. The `RWCalculator` class takes a number of optional parameters, for which see the API documentation.

Once we have instantiated a calculator object with our desired configuration, we pass the configured object to `rw.calculate`. Our configurations will be stored in `rw.metadata`.

The data calculated is stored in a pandas dataframe in `RollingWindows.result`, as shown below.

In [ ]:
rw.result.head()

## Plotting the Results of the Rolling Windows Analysis

Since `rw.result` is a pandas DataFrame, you can use the pandas [`DataFrame.plot.line`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.line.html) method to create a plot based on it with something like the code below.

In [ ]:
averages = rw.result.plot.line()

However, you may wish to generate plots with features specific to Rolling Windows analysis, or you may wish to adopt some plotting toolkit other than [matplotlib.pyplot.plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot), which is used by pandas. For this purpose, the `RollingWindows.plot()` method allows you to supply a plotter class.

Currently, there are two plotter classes, `RWSimplePlotter` (which produces static images) and `RWPlotlyPlotter` (which uses [Plotly](https://plotly.com/) to produce interactive images). In this notebook we will use `RWSimplePlotter`, and an example of how to use `RWPlotlyPlotter` is given at the end of the tutorial.

You can pass a plotter's string id to `RollingWindows.plot()` as shown below to call the class with the default configuration. By default, `RollingWindows.plot()` does not automatically display the output; if you want this behaviour, add `show=True`. The figure itself is stored in the `RollingWindows` object's `fig` attribute, so you can access it later by calling `rw.fig`

> [!NOTE] 
> `RWSimplePlotter.plot(show=True)` calls `matplotlib.pyplot.show`, which cannot display the figure in a Jupyter notebook (or similar) environment. The workaround is to make the extra call to `rw.fig`, as shown in the examples below.
>
> `RWPlotlyPlotter.plot(show=True)` _does_ work in notebook environments.

In [ ]:
from rollingwindows import plotters

rw.plot(plotter="rw_simple_plotter")
rw.fig

The main difference between the plot produced by the Lexos plotter and that produced by pandas is the default title and axis labels. You can change these easily by creating an instance of the plotter with a configuration and then passing the instance to `RollingWindows.plot()`, as shown below: 

In [ ]:
plotter = plotters.RWSimplePlotter(
    title="Rolling Averages of Words",
    xlabel="Word Count"
)
rw.plot(plotter=plotter)
rw.fig

See the API documentation for a complete list of options. As we will see, the main value of using a built-in plotter comes when you employ milestones in your document (see below).

## Filters

Filters are a way to reduce the token count in a doc without losing token attributes such as lemma or part-of-speech labels. In the code below, we apply the built-in word filter.

#### Why filter documents?

Lexos provides ways to preprocess a text to remove items like punctuation before the text is converted to a spaCy `Doc`. It is also possible to extract a list of desirable tokens in a `Doc` and then create a new `Doc` from this list. However, in both cases, the original position of the token will be lost, and this may affect token attributes such as part of speech, which you may want to access for your Rolling Windows analysis. A Rolling Windows filter instead creates a new, filtered `Doc` with the original position and token attributes preserved. 

In [ ]:
from rollingwindows import filters

rw.set_windows(1000, "tokens", filter="word_filter")

The example above will strip anything that is not a punctuation mark or a space from the document before generating windows. However, this may not be how you define a "word". You can configure the filter to, for instance, exclude digits by first creating an instance of the filter and then passing it to `RollingWindows.set_windows()`. This is shown in the below.

Note that you can inspect the result of your filter by accessing the `word_ids` property of your filter. This is a set of all token indexes in your original document that are retained as "words".


In [ ]:
from rollingwindows import filters

# Create a filter by instantiating the class
wordfilter = filters.WordFilter(rw.doc, exclude_digits=True)
rw.set_windows(1000, "tokens", filter=wordfilter)

print("First ten words:")
first_ten_words = list(wordfilter.word_ids)[0:10]
for i in first_ten_words:
    print(f"- {rw.doc[i]}")


The `WordFilter` class optionally takes the `exclude_digits`, `exclude_roman_numerals`, and `exclude_pattern` keywords to filter out tokens not identified as words. The last can take a string or regex pattern to match. You can also adjust which spaCy token attributes are retained in the filtered document. See the API documentation for further details.

Lexos currently has one other filter `NonStopwordFilter`. This can be used to remove all tokens that are labelled stopwords by the language model. And `additional_stopwords` parameter takes a list of additional stopwords and a `case_sensitive` parameter allows you to control whether or not the filter should match these stopwords regardless of capitalisation.  

## Milestones

Milestones are tokens within a spaCy `Doc` that have labels designating that they mark structural divisions (e.g. chapters) within the document. They are useful for Rolling Windows analysis because the can be displayed by a plotter, helping your to see where the peaks and valleys correspond to divisions within the text.

The `milestones` module was originally a component of the Lexos `cutter` module. In the next release of Lexos, a version of `milestones` compatible with Rolling Windows will be a separate module. Since that module is not available in the current release, `milestones` is for development purposes included as a submodule of `rollingwindows`.

To generate milestones, the standard procedure is to instantiate a `Milestones` object with a `Doc` and then match patterns in the `Doc` that you wish to identify as milestones. The matches are generated as a list of spaCy [`Span`](https://spacy.io/api/span) objects, and you can inspect them to see if they represent the desired tokens. In the code below, we list each match with its token index in the `Doc`.

In [ ]:
from rollingwindows import milestones

ms = milestones.Milestones(doc, nlp="en_core_web_sm")
spans = ms.get_matches(patterns=["Chapter"])
for span in spans:
    print(span.text, span.start)

If we are happy with our milestones, we can then assign them to the `Doc` with `Milestones.set_milestones()`. This adds two [spaCy custom token attributes](https://spacy.io/usage/processing-pipelines#custom-components-attributes): `milestone_iob` and `milestone_label`. Note that in spaCy, custom attributes must be referenced with the prefix `._.`. Below, we print the values of attributes for first ten tokens.

In [ ]:
ms.set_milestones(spans)

for token in doc[6:12]:
    print(token._.milestone_iob, token._.milestone_label)

The `milestone_iob` attribute indicates whether the token is inside, outside, or at the beginning of a milestone. Since we matched the single token "Chapter", it has the value "B", and all other tokens have the value "O". The `milestone_label` is an empty string or our matched token. If we matched multiple tokens some tokens would have `milestone_iob` values of "I", and milestone labels would show multiple tokens.

In the code below, we include a regex expression to match "Chapter" followed by a number. We use `mode="string"` to tell Lexos to search the raw text, rather than searching over tokens. See the API documentation for a full range of search modes. After getting the matches, we inspect the first two to obtain their start tokens. We then use this information to print the `milestone_iob` and `milestone_label` values for these and the surrounding tokens.

In [ ]:
ms.reset()
spans = ms.get_matches(patterns=["Chapter [0-9]+"], mode="string")

for span in spans[0:2]:
    print(span.text, span.start)

ms.set_milestones(spans)

print()

for token in doc[7:12]:
    print(token._.milestone_iob, token._.milestone_label)

print("...")

for token in doc[1005:1010]:
    print(token._.milestone_iob, token._.milestone_label)


The `Milestones` class has a handy `to_list()` method which provides access to other information:

In [ ]:
ms.to_list()

The `RWSimplePlotter` class accepts a list of milestone labels, where each label is a dictionary with the label as the key and the start character index (or start token index, if appropriate) as the value. We can easily produce this structure using `Milestones.to_list()`.

In [ ]:
milestone_labels = []

for milestone in ms.to_list():
    milestone_labels.append({milestone["text"]: milestone["start_token"]})

milestone_labels

We can now re-plot our Rolling Windows analysis using the milestone labels. The `show_milestones` parameter displays vertical lines at the index of the token with the `milestone_iob` "B" value. The `show_milestone_labels` parameter will display the milestone labels and lines at the locations in the `milestone_labels` list we created.

In [ ]:
plotter = plotters.RWSimplePlotter(
    title="Rolling Averages of Words",
    xlabel="Word Count",
    show_milestones=True,
    show_milestone_labels=True,
    milestone_labels=milestone_labels
)
rw.plot(plotter=plotter)
rw.fig

There are a number of options for manipulating how milestones and milestone labels are plotted. See the API documentation for the full range of possibilities.

## Plotting with Plotly

The `RWPlotlyPlotter` class uses [Plotly](https://plotly.com/) to output a similar line graph with interactive features. To the display the figure, you can either use `RollingWindows.plot(show=True)` or call `rw.fig` after running the function.

> [!IMPORTANT]
> Unlike `RWSimplePlotter`, `RWPlotlyPlotter` requires milestones to take the form of a dictionary with labels as keys and start token ids as values.

In [ ]:
# Create a milestone_labels dictionary
milestone_labels = {milestone["text"]: milestone["start_token"] for milestone in ms.to_list()}

# Configure the Plotly plotter
plotter = plotters.RWPlotlyPlotter(
    title="Rolling Averages of Words",
    xlabel="Word Count",
    ylabel="Average Frequency",
    show_milestones=True,
    show_milestone_labels=True,
    milestone_labels=milestone_labels,
    milestone_label_rotation=45,
)

# Create the plot
rw.plot(plotter=plotter, show=True)
# rw.fig

`RWPlotlyPlotter` is highly configurable. See the API Documentation for a full set of parameters available in the Lexos API.